In [10]:
%run ../../Utils/yp_utils.py

from functools import reduce

# Initial setup

In [2]:
paper_pmid = 28472365
paper_name = 'maclean_zhang_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheet_names = ['High Temp. (40C)','EtOH','H2O2','NaCl','CoCL2','SO']

In [9]:
original_data_list = []
for s in sheet_names:
    original_data = pd.read_excel('raw_data/supplementary_data_6.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data['Strain'].astype(str)
    
    # Remove underscore annotations
    original_data['orf'] = original_data['orf'].apply(lambda x: x.split('_')[0])
    
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = original_data.iloc[:,2].astype(float)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 4520 x 10
Empty DataFrame
Columns: [Strain, Fitness relative to HO in YPD, Fitness relative to HO in 40C, Ratio (40C/YPD), strain/HO read ratio in YPD1, strain/HO read ratio in YPD2, strain/HO ratio in 40C, SD, P-value, Q-value, orf]
Index: []
(4481, 1)
Original data dimensions: 4520 x 10
Empty DataFrame
Columns: [Strain, Fitness relative to HO in YPD, Fitness relative to HO in EtOH, Ratio (EtOH/YPD), strain/HO read ratio in YPD1, strain/HO read ratio in YPD2, strain/HO read ratio in EtOH, SD, P-value, Q-value, orf]
Index: []
(4481, 1)
Original data dimensions: 4520 x 10
Empty DataFrame
Columns: [Strain, Fitness relative to HO in YPD, Fitness relative to HO in H2O2, Ratio (H2O2/YPD), strain/HO read ratio in YPD1, strain/HO read ratio in YPD2, strain/HO read ratio in H2O2, SD, P-value, Q-value, orf]
Index: []
(4481, 1)
Original data dimensions: 4520 x 10
Empty DataFrame
Columns: [Strain, Fitness relative to HO in YPD, Fitness relative to HO in NaCl, Ratio (NaCl

In [15]:
original_data = reduce(lambda x, y: pd.merge(x, y, how='outer', on='orf'), original_data_list)


In [17]:
dataset_ids = [16395, 16397, 16399, 16396, 16400, 16398]

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,16395,16397,16399,16396,16400,16398
data_type,value,value,value,value,value,value
orf,,,,,,
YAL001C,0.955657,1.046975,0.947927,0.986013,0.902596,0.947244
YAL002W,0.836811,1.012243,1.017282,0.727588,0.836118,0.753172
YAL004W,0.970256,0.988769,1.023451,1.024669,0.965046,1.039818
YAL005C,0.951158,0.987666,1.006057,0.999740,0.962266,1.001685
YAL007C,0.983390,1.033139,0.989239,0.969792,0.930264,0.965912


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [24]:
data.head()

,dataset_id,16395,16397,16399,16396,16400,16398
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
1,YAL001C,0.955657,1.046975,0.947927,0.986013,0.902596,0.947244
2,YAL002W,0.836811,1.012243,1.017282,0.727588,0.836118,0.753172
1863,YAL004W,0.970256,0.988769,1.023451,1.024669,0.965046,1.039818
4,YAL005C,0.951158,0.987666,1.006057,0.999740,0.962266,1.001685
5,YAL007C,0.983390,1.033139,0.989239,0.969792,0.930264,0.965912


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [28]:
data_all.head()


,dataset_id,16395,16397,16399,16396,16400,16398,16395,16397,16399,16396,16400,16398
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
1,YAL001C,0.955657,1.046975,0.947927,0.986013,0.902596,0.947244,-0.228488,0.481811,-0.493745,0.147078,-0.904414,-0.382720
2,YAL002W,0.836811,1.012243,1.017282,0.727588,0.836118,0.753172,-1.314935,0.146515,0.278193,-3.149696,-1.583290,-2.457544
1863,YAL004W,0.970256,0.988769,1.023451,1.024669,0.965046,1.039818,-0.095027,-0.080096,0.346858,0.640221,-0.266676,0.606997
4,YAL005C,0.951158,0.987666,1.006057,0.999740,0.962266,1.001685,-0.269609,-0.090744,0.153255,0.322193,-0.295066,0.199308
5,YAL007C,0.983390,1.033139,0.989239,0.969792,0.930264,0.965912,0.025043,0.348245,-0.033926,-0.059861,-0.621873,-0.183142


# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 28472365...
Inserting the new data...


100%|██████████| 6/6 [00:38<00:00,  6.42s/it]

Updating the data_modified_on field...
